# AutoCAD MegaIntellegence Project


In [1]:

import win32com.client
import comtypes.client
import numpy as np
import pandas as pd
import math
import array

app = comtypes.client.GetActiveObject('AutoCAD.Application')
#app = win32com.client.Dispatch("AutoCAD.Application")
activedoc = app.ActiveDocument
msp =activedoc.ModelSpace

In [2]:
for blk in activedoc.Blocks:
    print(blk)
#    for ent in blk:
#         print(ent)

<POINTER(IAcadModelSpace) ptr=0x286debbf418 at 286e06de948>
<POINTER(IAcadPaperSpace) ptr=0x286dcd01b48 at 286e062e948>
<POINTER(IAcadBlock) ptr=0x286dcd36eb8 at 286e062eb48>
<POINTER(IAcadBlock) ptr=0x286dcd020f8 at 286e062ebc8>
<POINTER(IAcadBlock) ptr=0x286debbf098 at 286e062eac8>
<POINTER(IAcadBlock) ptr=0x286dcd01d78 at 286e06de948>
<POINTER(IAcadBlock) ptr=0x286debbefb8 at 286e062e6c8>
<POINTER(IAcadBlock) ptr=0x286dcd01988 at 286e06de948>
<POINTER(IAcadBlock) ptr=0x286dcd36b38 at 286e062e948>


In [134]:
# set variables
df = pd.DataFrame(columns ={'device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y'})
df = df[['device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y']]
dim_aligned, dim_rotated, blockref, i = 0,0,0,0
# view each object in ModelSpace
for item in msp:
    #TODO: need to check if that actually device block
    #if it's a device:
    if item.ObjectName == 'AcDbBlockReference':
        #find BLOCK object by given BlockRef
        block = activedoc.Blocks.Item(item.Name)
        print (f'Block name: {block}')
        for element in block:
            if element.Layer =="attachment point":
                # fill the dataframe with the coordinates of the attachment points
                  df = df.append({'device': i, \
                                'start_att_line_X': element.StartPoint[0] + item.InsertionPoint[0], \
                                'start_att_line_Y': element.StartPoint[1] + item.InsertionPoint[1], \
                                'end_att_line_X': element.EndPoint[0] + item.InsertionPoint[0],\
                                'end_att_line_Y': element.EndPoint[1] + item.InsertionPoint[1]},  ignore_index=True)
        i+=1
                
df['device'] = df['device'].astype(int)


Block name: <POINTER(IAcadBlock) ptr=0x286df215a08 at 286e27f06c8>
Block name: <POINTER(IAcadBlock) ptr=0x286df2151b8 at 286e1b16ac8>
Block name: <POINTER(IAcadBlock) ptr=0x286debbfc68 at 286e27f0848>


In [123]:
df

,device,end_att_line_X,end_att_line_Y,start_att_line_X,start_att_line_Y
0,0,473.643810,569.910725,468.643810,569.910725
1,0,471.143810,567.410725,471.143810,572.410725
2,0,543.643810,569.910725,538.643810,569.910725
3,0,541.143810,567.410725,541.143810,572.410725
4,0,473.643810,539.910725,468.643810,539.910725
5,0,471.143810,537.410725,471.143810,542.410725
6,0,543.643810,539.910725,538.643810,539.910725
7,0,541.143810,537.410725,541.143810,542.410725
8,1,661.799864,539.910725,656.799864,539.910725
9,1,661.799864,569.910725,656.799864,569.910725


In [5]:
# Length of attraction line must be 5mm
# Is the line in this row horisontal?
df['horizontal']= (df.iloc[:,1] == df.iloc[:,2])
# Find center of this line at axis X. If line is horizontal...
df['point_X'] =np.where(df['horizontal'], \
                        # any coordinate is suitable
                        df.iloc[:, 1], \
                        # otherwise, take the middle of the line by X
                        (np.minimum(df.iloc[:, 1], df.iloc[:, 2]) +2.5))

#same for Y
df['point_Y'] =np.where(~ df['horizontal'], \
                        df.iloc[:, 3], \
                        (np.minimum(df.iloc[:, 3], df.iloc[:, 4]) +2.5))    
#df

,device,point_X,point_Y
0,0,471.143810,569.910725
2,0,541.143810,569.910725
4,0,471.143810,539.910725
6,0,541.143810,539.910725
8,1,659.299864,539.910725
9,1,659.299864,569.910725
10,2,475.100512,460.108155
11,2,545.100512,460.108155


In [138]:
#df
    #print(f'got an item: {item}')
    #print(f'Object name: {item.ObjectName}')
    #print(item._methods_)
dims = []
for item in msp:    
    if item.ObjectName =='AcDbRotatedDimension':
        print('found rotated dimention.')
        print(item)
        #print(f'Object layer name: {item.Layer}\n')       
        dim_rotated = item
    elif item.ObjectName == 'AcDbAlignedDimension':
        print('found aligned dimention\n')
        dim_aligned = item
        dims.append(dim_aligned)
        
    #explode blocks
#    elif item.ObjectName == 'AcDbBlockReference':       
#         try:
#             print(f'Found a block.\n')
#             block = item
#             block.Explode()
#         except:
#             pass    # here we have some VARIANT bug. So just proceed.
#         i+=1

found rotated dimention.
<POINTER(IAcadDimRotated) ptr=0x286debbfd48 at 286e2256248>
found rotated dimention.
<POINTER(IAcadDimRotated) ptr=0x286deb6cd38 at 286e2256348>
found aligned dimention

found aligned dimention



In [140]:
df_dimensions = pd.DataFrame(columns ={'start_X', 'end_X','start_Y', 'end_Y'})
df_dimensions = df_dimensions[['start_X', 'end_X','start_Y', 'end_Y']]

for dim in dims:
    print(dim)

<POINTER(IAcadDimAligned) ptr=0x286dcd019f8 at 286e2256148>
<POINTER(IAcadDimAligned) ptr=0x286df268208 at 286e2256248>


In [128]:
# set variables
df3 = pd.DataFrame(columns ={'start_X', 'end_X','start_Y', 'end_Y'})
df3 = df3[['start_X', 'end_X','start_Y', 'end_Y']]



[<POINTER(IAcadDimAligned) ptr=0x286dcd019f8 at 286e2256148>,
 <POINTER(IAcadDimAligned) ptr=0x286df268208 at 286e2256248>]

In [113]:
sset

<POINTER(IAcadSelectionSets) ptr=0x286dcd36ba8 at 286e2238dc8>

In [94]:
# Довольно неочевидно, как взрывать размеры, не используя vba или lisp. Здесь надо будет добыть имя блока размера для метода explode.
# for item in msp:
#     if item.ObjectName == 'AcAlignedDimension':
#         #find BLOCK object by given BlockRef
#         activedoc.Blocks.Item(item.     )
#         #item.Explode(item.ObjectID)

#         print (item.ObjectID) 

In [97]:
dim_aligned.ExtLine1Point


(596.4774206594807, 603.8583661593816, 0.0)

Использовать фичу DIMATFIT 

вариант добычи точек привязки rotated размера - экспорт в dwf https://adn-cis.org/forum/index.php?topic=8448.msg31681#msg31681
И не забыть SelectionSet

In [ ]:
def FindAttachmentPoint(data):
    """This function converts given data to the coordinates of attachment points. 
    
    data: dataframe with coordinates of lines in layer 'attachment point'
    
    return
    """
    #FindAttachmentPoint(df[df['device']==device])
    print(f'data:\n{data}')
    print(data.iloc[[0]])


def MakeDimVertical(msp, point1, point2, loc):
    """This function make vertical dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass
    

def MakeDimHorizontal(msp, point1, point2, loc):
    """This function make horizontal dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass

def ProcessAttachmentPoints(msp, objects):
    """This function make coorginates of attachment points.
    
    msp:  model space
    objects:  lines from specified layer
    """
    pass


In [ ]:
pt = array.array('d', [0,0,0]) # to convert in variant
point = mSp.AddPoint(pt)
# Add a LINE
pt1 = array.array('d', [0.0,0.0,0]) # start point
pt2 = array.array('d', [20.0,20.0,0]) # end point
line = mSp.AddLine(pt1, pt2)

In [ ]:
s1 =  array.array('d', [500.0, 800.0, 0.0])
s2 =  array.array('d', [500.0, 1200.0, 0.0])
loc = array.array('d', [500.0, 1000.0, 0.0])
new_dim = mSp.AddDimAligned(s1, s2, loc)
new_dim.update()

In [ ]:
dim.


In [ ]:
df = pd.read_excel('E://Python/RubinProject/Journal.xlsx')

In [ ]:
make_dim_vertical()

In [ ]:
df = df.rename({'Имя прибора': 'prib_name',
               'Индекс по схеме': 'shema_index'},
               axis=1)

In [ ]:
# import win32com.client
# acad = win32com.client.Dispatch("AutoCAD.Application")

# doc = acad.ActiveDocument   # Document object


# # iterate trough all objects (entities) in the currently opened drawing
# # and if its a BlockReference, display its attributes and some other things.
# for entity in acad.ActiveDocument.ModelSpace:
#     name = entity.EntityName
#     if name == 'AcDbBlockReference':
#         HasAttributes = entity.HasAttributes
#         if HasAttributes:
#             print(entity.Name)
#             print(entity.Layer)
#             print(entity.ObjectID)
#             for attrib in entity.GetAttributes():
#                 print("  {}: {}".format(attrib.TagString, attrib.TextString))
                
#                 # update text
#                 attrib.TextString = 'modified with python'
#                 attrib.Update()

Фичи размеров в качестве таргетов:
- TextPosition (rotated размеры тоже их имеют)

Запасной вариант- взрывать кхерам все размеры, вытаскивать параметры линий, и таким образом получать координаты точек привязки размера, а заодно и координаты текста. Наверное, придётся так и сделать, чтобы обеспечить непересекаемоть размеров и других линий чертежа, типа линий габаритки

todo:

точка крепления может быть не посередине линии соответствующего слоя, а где-то на пересечении с линией прибора. Сделать проверку на небольшое отклонение и выдать сообщение чтобы поправили чертёж